## Setup

In [1]:
# Connect using pyodbc, sqlalchemy, and pandas
import sqlalchemy
import numpy as np
import pandas as pd

server = "sqlsvr-0092-mdp-02.85f8a2f57eaf.database.windows.net"
database = "Staging"
username = "pisrc-inkoo"
password = input("Enter database password: ")
driver = "ODBC Driver 17 for SQL Server"

engine = sqlalchemy.create_engine(
    f"mssql+pyodbc://{username}:{password}@{server}/{database}?driver={driver.replace(' ', '+')}"
)


def query_db(query, params=None):
    return pd.read_sql(query, engine, params=params)


df = query_db("SELECT @@version;")
print(df[""][0])


Microsoft SQL Azure (RTM) - 12.0.2000.8 
	Apr 18 2022 13:01:43 
	Copyright (C) 2021 Microsoft Corporation



# Binge Experience & PDF Analytics

In [2]:
# Columns of interest
query_db(
    """
    SELECT TOP 5
        SessionVisitorId,
        VisitStartDateTime,
        EventList,
        PageURL,
        PageName,
        EloquaContactId,
        UTM_Source,
        UTM_Medium,
        UTM_Campaign,
        UTM_Content,
        UTM_Term
        BingeId,
        BingeName,
        BingeCampaignId,
        BingeCriticalScore,
        BingeScoredAsset,
        BingeScoredAssetPath,
        BingeScoredAssetScore,
        PDFurl,
        PDFtitle,
        PDFpagecount
    FROM aem.RawTraffic
    WHERE VisitStartDateTime > '2022-06-15'
        AND BingeId <> '';
    """
)

,SessionVisitorId,VisitStartDateTime,EventList,PageURL,PageName,EloquaContactId,UTM_Source,UTM_Medium,UTM_Campaign,UTM_Content,BingeId,BingeName,BingeCampaignId,BingeCriticalScore,BingeScoredAsset,BingeScoredAssetPath,BingeScoredAssetScore,PDFurl,PDFtitle,PDFpagecount
0,583719550121999618_121251133910496911,2022-06-15 01:35:17,"215,216=27.00,10035,10036,10037,10038,10039,113",https://www.rockwellautomation.com/search/ra-z...,rockwellautomation:search,,,,,,,Basic Functionality Labs,builder_labs,5.0,,,,/content/dam/rockwell-automation/sites/downloa...,null,121
1,7155208166619660672_5588629153287961608,2022-06-15 09:03:27,"215,216=828.00,10035,10036,10037,10038,10039,113",https://www.rockwellautomation.com/search/ra-e...,rockwellautomation:search,,,,,,,Basic Functionality Labs,builder_labs,5.0,,,,/content/dam/rockwell-automation/sites/downloa...,"Getting Started in IAB (85 minutes, 121 pages)",121
2,4405954664442376543_7269107232429611232,2022-06-15 11:46:18,"200,249,100,101,105,111,112,119,120,124,129,14...",https://www.rockwellautomation.com/search/ra-e...,rockwellautomation:search,CRACP000000213285,,,,,,Basic Functionality Labs,builder_labs,5.0,,,,,,
3,4405954664442376543_7269107232429611232,2022-06-15 11:46:18,"201=5194.00,202=846.00,203=844.00,204=6445.00,...",https://www.rockwellautomation.com/search/ra-e...,rockwellautomation:search,CRACP000000213285,,,,,,Basic Functionality Labs,builder_labs,5.0,,,,,,
4,4405954664442376543_7269107232429611232,2022-06-15 11:46:18,"273,174,175,176,113",https://www.rockwellautomation.com/search/ra-e...,rockwellautomation:search,CRACP000000213285,,,,,,Basic Functionality Labs,builder_labs,5.0,,,,,,


# Lead Status

In [4]:
# ra_leadstage possible values
query_db(
    """
    SELECT DISTINCT
        ra_leadstage,
        ra_leadstagename
    FROM crm.Lead
    ORDER BY ra_leadstage;
    """
)


,ra_leadstage,ra_leadstagename
0,NaN,None
1,1.0,Unassigned
2,2.0,Awaiting Tele Acceptance
3,3.0,Awaiting Tele Qualification
4,4.0,Awaiting Sales Acceptance
5,5.0,Awaiting Sales Qualification
6,6.0,Distributor Lead
7,7.0,Qualified
8,8.0,External Lead


In [5]:
# statecode possible values
query_db(
    """
    SELECT DISTINCT
        statecode,
        statecodename
    FROM crm.Lead
    ORDER BY statecode;
    """
)


,statecode,statecodename
0,0,Open
1,1,Qualified
2,2,Disqualified


In [7]:
# Get latest lead status
query_db(
    """
    SELECT TOP 5
        c1.emailaddress1,
        c1.ra_leadstage,
        c1.ra_leadstagename,
        c1.statecode,
        c1.statecodename,
        c1.statuscode,
        c1.statuscodename
    FROM crm.Lead AS c1
    INNER JOIN (
        SELECT
            emailaddress1,
            MAX(createdon) AS createdon
        FROM crm.Lead
        GROUP BY emailaddress1
    ) AS c2
    ON c1.emailaddress1 = c2.emailaddress1
        AND c1.createdon = c2.createdon;
    """
)


,emailaddress1,ra_leadstage,ra_leadstagename,statecode,statecodename,statuscode,statuscodename
0,sherman.wagner@silganpfc.com,4,Awaiting Sales Acceptance,0,Open,1,New
1,miguel.chavez2@sbdinc.com,4,Awaiting Sales Acceptance,0,Open,1,New
2,jpierce@wlgore.com,5,Awaiting Sales Qualification,2,Disqualified,953810014,Duplicate Lead
3,timothy.smith@adient.com,4,Awaiting Sales Acceptance,0,Open,1,New
4,johnpatrick@rdm.net,4,Awaiting Sales Acceptance,0,Open,1,New


# Joins

In [8]:
# aem.RawTraffic: provides analytics data (key: EloquaContactID)
# crm.Lead: provides lead status (key: emailaddress1)
# elq.Contact: links the other two together (has both EloquaContactId and EmailAddress)

query_db(
    """
    WITH cl AS (
        SELECT
            c1.emailaddress1,
            c1.ra_leadstage,
            c1.statecode
        FROM crm.Lead AS c1
        INNER JOIN (
            SELECT
                emailaddress1,
                MAX(createdon) AS createdon
            FROM crm.Lead
            GROUP BY emailaddress1
        ) AS c2
        ON c1.emailaddress1 = c2.emailaddress1
            AND c1.createdon = c2.createdon
    )

    SELECT TOP 5
        ec.EmailAddress,
        cl.ra_leadstage,
        ar.PageName,
        ar.PageURL,
        ar.BingeId,
        ar.BingeName,
        ar.BingeCampaignId,
        ar.BingeCriticalScore,
        ar.BingeScoredAsset,
        ar.BingeScoredAssetPath,
        ar.BingeScoredAssetScore
    FROM
        aem.RawTraffic AS ar,
        cl,
        elq.Contact AS ec
    WHERE ar.EloquaContactId = ec.EloquaContactId
        AND cl.emailaddress1 = ec.EmailAddress
        AND ar.VisitStartDateTime > '2022-06-12'
        AND ar.BingeId <> '';
    """
)

,EmailAddress,ra_leadstage,PageName,PageURL,BingeId,BingeName,BingeCampaignId,BingeCriticalScore,BingeScoredAsset,BingeScoredAssetPath,BingeScoredAssetScore
0,cristobal.escoto@ecnautomation.com,5,rockwellautomation:search,https://www.rockwellautomation.com/search/ra-e...,be1650550449644850,Basic Functionality Labs,builder_labs,5.0,,,
1,cristobal.escoto@ecnautomation.com,5,rockwellautomation:search,https://www.rockwellautomation.com/search/ra-e...,be1650550449644850,Basic Functionality Labs,builder_labs,5.0,,,
2,cristobal.escoto@ecnautomation.com,5,rockwellautomation:search,https://www.rockwellautomation.com/search/ra-e...,be1650550449644850,Basic Functionality Labs,builder_labs,5.0,,,
3,cristobal.escoto@ecnautomation.com,5,rockwellautomation:search,https://www.rockwellautomation.com/search/ra-e...,be1650550449644850,Basic Functionality Labs,builder_labs,5.0,,,
4,cristobal.escoto@ecnautomation.com,5,rockwellautomation:search,https://www.rockwellautomation.com/search/ra-e...,be1650550449644850,Basic Functionality Labs,builder_labs,5.0,,,


In [9]:
query_db(
    """
    SELECT DISTINCT
        BingeScoredAsset,
        BingeScoredAssetPath
    FROM aem.RawTraffic
    WHERE VisitStartDateTime > '2022-06-01'
    """
)


,BingeScoredAsset,BingeScoredAssetPath
0,Electric Vehicles And Smart Manufacturing - A ...,/content/dam/rockwell-automation/sites/downloa...
1,None,None
2,,
3,"Project Design Assistant (30 minutes, 30 pages)",/content/dam/rockwell-automation/sites/downloa...
4,"Getting Started in IAB (85 minutes, 121 pages)",/content/dam/rockwell-automation/sites/downloa...
